In [1232]:
import os
import pandas as pd
from fastbook import *
from fastai.tabular.all import *
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from dtreeviz.trees import *
import sklearn

def r_mse(pred,y): return round(math.sqrt(((pred-y)**2).mean()), 6)
def m_rmse(m, xs, y): return r_mse(m.predict(xs), y)

traindf = pd.read_csv('/mnt/c/Users/jsult/Desktop/spacetitanic3/train.csv',low_memory=False)
testdf = pd.read_csv('/mnt/c/Users/jsult/Desktop/spacetitanic3/test.csv',low_memory= False)
traindf['Set'] = 'Train'
testdf['Set'] = 'Test'
togeth = pd.concat([testdf,traindf])




def split_group_number(df):
    df['Group'] = df['PassengerId'].apply(lambda x: x.split("_")[0] if pd.notna(x) else pd.NA)
    df.Group = df.Group.astype('float')
    df['GroupNumber'] = df['PassengerId'].apply(lambda x: x.split("_")[1] if pd.notna(x) else pd.NA)
    df.GroupNumber = df.GroupNumber.astype('float')
    return df
togeth = split_group_number(togeth)


def split_cabin(df):
    df['CabinDeck'] = df['Cabin'].apply(lambda x: x.split("/" )[0] if pd.notna(x) else pd.NA)
    custom_order = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T']
    df['CabinDeck'] = pd.Categorical(df['CabinDeck'], categories=custom_order, ordered=True)
    df = df.sort_values(by='CabinDeck')
    df['CabinSide'] = df['Cabin'].apply(lambda x: x.split("/")[2] if pd.notna(x) else pd.NA)
    df['CabinNum'] = df['Cabin'].apply(lambda x: int(x.split("/")[1]) if pd.notna(x) else pd.NA)
    return df
togeth = split_cabin(togeth)


def split_cabin(df):
    df['CabinDeck'] = df['Cabin'].apply(lambda x: x.split("/" )[0] if pd.notna(x) else pd.NA)
    custom_order = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T']
    df['CabinDeck'] = pd.Categorical(df['CabinDeck'], categories=custom_order, ordered=True)
    df = df.sort_values(by='CabinDeck')
    df['CabinSide'] = df['Cabin'].apply(lambda x: x.split("/")[2] if pd.notna(x) else pd.NA)
    df['CabinNum'] = df['Cabin'].apply(lambda x: int(x.split("/")[1]) if pd.notna(x) else pd.NA)
    return df
togeth = split_cabin(togeth)



In [1203]:
len(togeth[togeth.Group == '0001'])

SyntaxError: invalid syntax (1889472585.py, line 1)

# ['6612', '9223'] both have multiple cabins missing in group


NEED to make a list rather than the last missing and pop them along, in case two are missing before the next skip

In [1204]:
togeth = togeth.sort_values(by = ['Group','GroupNumber'])
togeth.reset_index(inplace = True)
togeth.drop(columns = 'index',inplace = True)

heather = togeth.copy()



def check(dic,row):
    df = togeth.iloc[row[0]:]
    for key in dict.keys():
        if len(df[(df.CabinNum == dic[key][0] + 1) & (df.CabinSide == 'P') & (df.CabinDeck == key)]) == 0:
            print('failedkey',key)
            print("CabinNum",key,(df.CabinNum == dic[key][0] + 1).sum())
            print("CabinSide","P",(df.CabinSide == 'P').sum())
            print("cabindeck",key,(df.CabinDeck == key).sum())
            print(df[(df.CabinNum == dic[key][0] + 1) & (df.CabinSide == 'P') & (df.CabinDeck == key)])
            return [False]
        if len(df[(df.CabinNum == dic[key][1] + 1) & (df.CabinSide == 'S') & (df.CabinDeck == key)]) == 0:
            print(df[(df.CabinNum == dic[key][0] + 1) & (df.CabinSide == 'S') & (df.CabinDeck == key)])
            print('failedkey',key)
            return [False]
    return [True]

        
dict = {}
for deck in list(togeth.CabinDeck.unique())[:-1]:
    dict[deck] = [-1,-1]
    

cabinsidemap = {'P' : 1, 'S' : 0}


def fillcabin(length):
    missing = []
    pos = 0 
    j = -1
    for row in togeth.iterrows():
        if j < length:
            j +=1

            print(" ")
            print(j)
            print("missing",missing)
            print("pos",pos)
            
            # missing cabin
            
            if pd.isna(row[1].Cabin):

                print('IS MISSING !!')
                print('IS MISSING !!')
                print('IS MISSING !!')
                print( row[0])
               
                aboverow = togeth.iloc[row[0] + 1]
                side = cabinsidemap[aboverow.CabinSide]
                
                print("NA Cabin")
                                
                if len(togeth[togeth.Group == row[1].Group]) == 1:
                    print("sologroup")
                    missing.append(row[0])     
                #if pos < len(missing):
                elif pos == len(missing):
                    print('no missing cabins to fill')
                    togeth.loc[row[0], 'Cabin'] = 'holder'


                    
                elif (aboverow.Group == row[1].Group) and (dict[aboverow.CabinDeck][side] == aboverow.CabinNum - 1):
                    print("beforesamecabin")
                    print(row[0])
                    togeth.loc[row[0],'Cabin'] = aboverow.Cabin

                elif(row[1].Group in list(togeth.sort_values(by = ['Group','GroupNumber']).iloc[:row[0]].Group)):
                    print("isna seen group")
                    if check(dict,row)[0]:
                        print("isna checked")
                        togeth.loc[row[0], 'Cabin'] = 'same'

                else:
                    print("isna not seen group")
                    missing.append(row[0])


            #not missing cabin
            if not pd.isna(row[1].Cabin):
                #missing gap
                side = cabinsidemap[row[1].CabinSide]
                if dict[row[1].CabinDeck][side] + 1 < row[1].CabinNum:
                    print('GAP')
                    print('GAP')
                    print('GAP')
                    print(row[1].Cabin)
                    print(dict)
                    print("gap found from", row[1].Cabin)
                    togeth.loc[missing[pos], 'Cabin'] = row[1].CabinDeck + '/' + str(row[1].CabinNum - 1) + '/' + row[1].CabinSide
                    pos += 1
                    dict[row[1].CabinDeck][side] += 2
                    
                #next step
                elif dict[row[1].CabinDeck][side] + 1 == row[1].CabinNum:
                    dict[row[1].CabinDeck][side] += 1
                                         


        

            
fillcabin(450)
    

 
0
missing []
pos 0
 
1
missing []
pos 0
 
2
missing []
pos 0
 
3
missing []
pos 0
 
4
missing []
pos 0
 
5
missing []
pos 0
 
6
missing []
pos 0
 
7
missing []
pos 0
 
8
missing []
pos 0
 
9
missing []
pos 0
 
10
missing []
pos 0
 
11
missing []
pos 0
 
12
missing []
pos 0
 
13
missing []
pos 0
 
14
missing []
pos 0
 
15
missing []
pos 0
IS MISSING !!
IS MISSING !!
IS MISSING !!
15
NA Cabin
sologroup
 
16
missing [15]
pos 0
GAP
GAP
GAP
G/3/S
{'B': [-1, 1], 'F': [3, 2], 'A': [0, -1], 'G': [1, -1], nan: [-1, -1], 'C': [-1, -1], 'E': [-1, -1], 'D': [-1, -1]}
gap found from G/3/S
 
17
missing [15]
pos 1
 
18
missing [15]
pos 1
 
19
missing [15]
pos 1
 
20
missing [15]
pos 1
 
21
missing [15]
pos 1
 
22
missing [15]
pos 1
 
23
missing [15]
pos 1
 
24
missing [15]
pos 1
 
25
missing [15]
pos 1
 
26
missing [15]
pos 1
 
27
missing [15]
pos 1
 
28
missing [15]
pos 1
 
29
missing [15]
pos 1
 
30
missing [15]
pos 1
 
31
missing [15]
pos 1
 
32
missing [15]
pos 1
 
33
missing [15]
pos 1
 
34
mi

In [1163]:
togeth.loc[15,'Cabin']

'G/2/S'

In [1229]:
i = 0

In [1230]:
i -= 20

In [1231]:
i += 20
heather.iloc[i:i+20][['Cabin','CabinDeck','CabinNum','Group','GroupNumber','HomePlanet']]

,Cabin,CabinDeck,CabinNum,Group,GroupNumber,HomePlanet
0,B/0/P,B,0,1.0,1.0,Europa
1,F/0/S,F,0,2.0,1.0,Earth
2,A/0/S,A,0,3.0,1.0,Europa
3,A/0/S,A,0,3.0,2.0,Europa
4,F/1/S,F,1,4.0,1.0,Earth
5,F/0/P,F,0,5.0,1.0,Earth
6,F/2/S,F,2,6.0,1.0,Earth
7,G/0/S,G,0,6.0,2.0,Earth
8,F/3/S,F,3,7.0,1.0,Earth
9,B/1/P,B,1,8.0,1.0,Europa


In [810]:
togeth.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Set,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum
693,1422_03,Europa,False,A/15/S,55 Cancri e,48.0,False,58.0,105.0,0.0,7536.0,103.0,Grasath Qualelfic,Test,NaN,1422.0,3.0,A,S,15
5035,5384_01,Europa,False,A/65/S,TRAPPIST-1e,34.0,True,0.0,5210.0,0.0,1332.0,2.0,Polam Dinger,Train,True,5384.0,1.0,A,S,65
3407,3667_01,Europa,True,A/42/S,55 Cancri e,26.0,False,0.0,0.0,0.0,0.0,0.0,Marinon Bewarsecal,Train,True,3667.0,1.0,A,S,42
6989,7431_01,Europa,False,A/90/S,55 Cancri e,31.0,False,NaN,2954.0,0.0,170.0,756.0,Altanin Barmant,Train,False,7431.0,1.0,A,S,90
2859,3088_01,Europa,False,A/25/P,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,0.0,Dscha Dishearly,Train,False,3088.0,1.0,A,P,25


In [787]:
togeth[togeth.Cabin == 'E/16/S']

,index,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Set,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum
356,242,0260_01,Europa,False,E/16/S,TRAPPIST-1e,26.0,False,0.0,4810.0,0.0,981.0,2054.0,Segino Inderly,Train,False,260.0,1.0,E,S,16


In [751]:
togeth.Cabin.isna().sum()

284

In [896]:
togeth[togeth.Cabin.isna()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Set,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum
404,0293_01,Europa,True,NaN,TRAPPIST-1e,47.0,False,0.0,0.0,0.0,0.0,0.0,Tauxon Suptibler,Test,NaN,293.0,1.0,NaN,NaN,NaN
412,0301_01,Earth,True,NaN,TRAPPIST-1e,16.0,False,0.0,0.0,0.0,0.0,0.0,Margia Wriggins,Train,False,301.0,1.0,NaN,NaN,NaN
421,0310_01,Europa,False,NaN,TRAPPIST-1e,67.0,False,NaN,230.0,0.0,4476.0,241.0,Naviton Coudered,Train,False,310.0,1.0,NaN,NaN,NaN
440,0323_01,Earth,True,NaN,55 Cancri e,18.0,False,0.0,0.0,0.0,0.0,0.0,Joyn Gaineyerson,Test,NaN,323.0,1.0,NaN,NaN,NaN
444,0327_01,Earth,False,NaN,TRAPPIST-1e,19.0,False,17.0,0.0,784.0,0.0,2.0,Jasony Ocherman,Train,True,327.0,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12892,9223_01,Mars,True,NaN,TRAPPIST-1e,24.0,False,0.0,0.0,NaN,0.0,0.0,Weessh Sun,Test,NaN,9223.0,1.0,NaN,NaN,NaN
12893,9223_02,Mars,True,NaN,TRAPPIST-1e,17.0,False,0.0,0.0,0.0,0.0,0.0,Perit Sun,Test,NaN,9223.0,2.0,NaN,NaN,NaN
12909,9233_01,Earth,False,NaN,TRAPPIST-1e,41.0,False,3.0,0.0,668.0,0.0,0.0,Royce Giller,Train,True,9233.0,1.0,NaN,NaN,NaN
12918,9238_05,Earth,True,NaN,TRAPPIST-1e,14.0,False,0.0,0.0,0.0,0.0,0.0,Caseye Emenez,Test,NaN,9238.0,5.0,NaN,NaN,NaN


In [1148]:
deckdict = {}
deckdictside = {'P':0,'S':1}

togeth = togeth.sort_values(by = ['Group','GroupNumber'])
togeth.reset_index(inplace = True)
togeth.drop(columns = 'index',inplace = True)


miss = []

for deck in list(togeth.CabinDeck.unique()):
    deckdict[deck] = [-1,-1]
    
i = 0

for cabin in list(togeth.Cabin.dropna().unique()):
    if i < 500:
        i += 1
        cabdeck = cabin.split("/")[0]
        cabinside = cabin.split("/")[2]
        cabside = deckdictside[cabinside]
        cabnum = int(cabin.split("/")[1])
        
        newcabnum = str(cabnum - 1)

        if deckdict[cabdeck][cabside] + 2 == cabnum:
            missingcabin = cabdeck + '/' + newcabnum + '/' + cabinside
            miss.append((missingcabin,(str(togeth[togeth.Cabin == cabin].iloc[0].name) + ",")))
            deckdict[cabdeck][cabside] += 2
            
        elif deckdict[cabdeck][cabside] + 2 < cabnum:
            missingcabin = cabdeck + '/'+ newcabnum + '/'+ cabinside
            miss.append((missingcabin,(str(togeth[togeth.Cabin == cabin].iloc[0].name) + ",")))
            print("problem",cabin)
            deckdict[cabdeck][cabside] += 3

        elif deckdict[cabdeck][cabside] + 1 == cabnum:
                        deckdict[cabdeck][cabside] += 1

problem E/22/P


In [1149]:
miss

[('G/2/S', '16,'),
 ('F/20/P', '143,'),
 ('G/39/P', '327,'),
 ('E/15/S', '356,'),
 ('G/40/S', '414,'),
 ('G/46/S', '443,'),
 ('F/70/P', '445,'),
 ('B/13/P', '446,'),
 ('C/13/S', '447,'),
 ('G/49/S', '495,'),
 ('E/21/P', '518,'),
 ('B/12/S', '604,')]

In [1150]:
i = -20

In [1157]:
for row in togeth[:500].iterrows():
    if pd.isna(row[1].Cabin) and len(togeth[togeth.Group == row[1].Group]) == 1:
        print(row[0])
    

15
137
315
331
404
412
421
440
444
492


In [1153]:
len(togeth[togeth.Group == row[1].Group])

2

In [1158]:
i = 380

In [1159]:
i+= 20
togeth[['Cabin','Group','GroupNumber']][i:i+20]

,Cabin,Group,GroupNumber
400,B/7/S,290.0,3.0
401,B/7/S,290.0,4.0
402,D/10/S,291.0,1.0
403,F/56/S,292.0,1.0
404,NaN,293.0,1.0
405,B/8/S,294.0,1.0
406,F/66/P,295.0,1.0
407,F/57/S,296.0,1.0
408,G/47/P,297.0,1.0
409,A/3/P,298.0,1.0


In [1104]:
togeth[togeth.Cabin == 'G/45/S']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Set,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum
439,0322_01,Earth,True,G/45/S,PSO J318.5-22,13.0,False,0.0,0.0,0.0,0.0,0.0,Annya Blange,Train,False,322.0,1.0,G,S,45


In [1052]:
a = pd.DataFrame(miss)

In [1055]:
b = [str(x) + "," for x in list(a[1])]

In [1056]:
a[1] = b

In [1063]:
miss

[('G/2/S', '16,'),
 ('F/20/P', '143,'),
 ('G/39/P', '327,'),
 ('E/15/S', '356,'),
 ('G/40/S', '414,'),
 ('G/46/S', '443,'),
 ('F/70/P', '445,'),
 ('B/13/P', '446,'),
 ('C/13/S', '447,'),
 ('G/49/S', '495,'),
 ('E/21/P', '518,'),
 ('B/12/S', '604,'),
 ('G/73/P', '667,'),
 ('E/30/S', '704,'),
 ('G/75/S', '717,'),
 ('G/79/S', '771,'),
 ('F/106/S', '811,'),
 ('B/30/P', '1018,'),
 ('F/152/P', '1043,'),
 ('G/110/S', '1056,'),
 ('F/157/S', '1176,'),
 ('G/121/P', '1187,'),
 ('E/49/P', '1245,'),
 ('F/181/P', '1254,'),
 ('C/35/S', '1265,'),
 ('F/170/S', '1324,'),
 ('G/151/P', '1444,'),
 ('E/58/P', '1494,'),
 ('D/36/S', '1546,'),
 ('C/40/S', '1555,'),
 ('F/213/S', '1582,'),
 ('E/65/P', '1611,'),
 ('G/171/P', '1612,'),
 ('F/230/S', '1720,'),
 ('F/235/S', '1767,'),
 ('E/81/P', '1853,'),
 ('F/293/P', '2149,'),
 ('F/300/S', '2224,'),
 ('F/307/P', '2252,'),
 ('F/320/P', '2312,'),
 ('G/257/S', '2362,'),
 ('G/273/P', '2426,'),
 ('E/104/P', '2455,'),
 ('G/267/S', '2482,'),
 ('G/280/S', '2539,'),
 ('F/350

In [1064]:
pd.DataFrame(a).to_csv('missincabin.csv')

In [1243]:
togeth[(togeth.Cabin.isna()) & (togeth.Set == 'Train')].head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Set,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum
15,0012_01,Earth,False,NaN,TRAPPIST-1e,31.0,False,32.0,0.0,876.0,0.0,0.0,Justie Pooles,Train,False,12.0,1.0,NaN,<NA>,<NA>
93,0101_01,Mars,True,NaN,TRAPPIST-1e,31.0,False,0.0,0.0,0.0,0.0,0.0,Book Trad,Train,True,101.0,1.0,NaN,<NA>,<NA>
103,0110_01,Europa,False,NaN,TRAPPIST-1e,32.0,False,0.0,410.0,6.0,3929.0,764.0,Graviph Aloubtled,Train,False,110.0,1.0,NaN,<NA>,<NA>
222,0239_01,Mars,False,NaN,TRAPPIST-1e,37.0,False,637.0,0.0,0.0,92.0,319.0,Diedow Resty,Train,False,239.0,1.0,NaN,<NA>,<NA>
227,0244_01,Mars,True,NaN,TRAPPIST-1e,43.0,False,0.0,0.0,0.0,0.0,0.0,Froos Sad,Train,True,244.0,1.0,NaN,<NA>,<NA>


In [1234]:
cleaned = pd.read_csv('/mnt/c/users/jsult/desktop/spacetitanic3/Cleaned and imputed data 2.csv',low_memory= False)


In [1250]:
cleaned[cleaned.Cabin == 'G/2/S']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Grouped,Deck,Side,Has_expenses,Is_Embryo


In [1244]:
cleaned[cleaned.PassengerId == '0012_01']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Grouped,Deck,Side,Has_expenses,Is_Embryo
15,0012_01,Earth,False,F/2/P,TRAPPIST-1e,31.0,False,32.0,0.0,876.0,0.0,0.0,False,False,F,P,True,False
